In [1]:
import os   #These two are useful modules
import sys

import numpy as np #Numerical Python

In [2]:
# Let's define te directory where you are with the "get current working directory" command...
work_dir = os.getcwd()
print("I am in dir %s" % work_dir)

#We can go up by using the path.dirname
main_dir =os.path.dirname(work_dir)
print("Main directory is %s" % main_dir)

#We can then define the data directory using the os.path.join command
data_dir = os.path.join(main_dir,"Data")
print("data dir is %s" % data_dir)

#..and we can define some output directory with the results
results_dir = os.path.join(main_dir,"results")

#if the directory does not exist, you can create it...
if not os.path.exists(results_dir):
    os.mkdir(results_dir)
    print("Directory %s created !" % results_dir)

I am in dir /home/jovyan/work/grlmrc/PLS/Code
Main directory is /home/jovyan/work/grlmrc/PLS
data dir is /home/jovyan/work/grlmrc/PLS/Data


### READS FILE FROM dati.tsv
Theese are the datas from Univerity: 1992 students

In [16]:
#define the filename, in this case is in the hw-data/hw01-data directory
data_filename = "dati.tsv"
data_file = open(os.path.join(data_dir,data_filename),"r")  #r means reading mode

#the readlines function gives a list with a line each
lines = data_file.readlines()
#we can create an output list with only the lines that we want
out_list=[]
#we can create an output list with only the lines that we want
out_list=[]
for li in range(len(lines)):
    line = lines[li].strip("\n")
    out_list.append(line)
header="COD_FIS,GENERE,PROV_RES_ATTUALE,COM_RES_ATTUALE,RES_PREC_SIGLA,RES_PREC_DES,DIPARTIMENTO,TIPO_CORSO_COD,TIPO_CORSO_DES,CDS_ID,CDS_COD,AA_IMM1,TIPO_TITOLO_SUP_DESC,TIPO_TITOLO_STRA_DESC,SCUOLA_DESC,COMUNE_SCUOLA_DESC,PROVINCIA_SCUOLA_SIGLA,VOTO_MATURITA,VOTO_MIN,VOTO_MAX,STA_STU_COD,STATO_STU_DESC,MOT_STASTU_COD,MOTIVO_STATO_STU_DESC,CDS_PROV,AA_ARRIVO_FISL,CDS_ARR,AA_PARTENZA_FISL,CDS_COD_L,CDS_DESC_L,VOTO_FINALE,LODE_FLG,ANNO_CT,DATA_CT,AA_ID,COD,DES,VOTO,LODE_FLG_1,CREDITI,DATA_SUP,AA_SUP_ID,RIC_ID,STA_SCE_COD,PRESIDENTE_COMMISSIONE"
column_names = header.split(",")

#create and init a dictionary of lists..
data_dict={}
for ci in range(len(column_names)):
    data_dict[column_names[ci]]=[]
    
#we start from 1 since we want to skip the first line containing the headers.
for ri in range(0,len(out_list)-1):
    if ri >=13496: data_line = out_list[ri+1].split("\t")
    else: data_line = out_list[ri].split("\t")
    if(ri==len(out_list)-2): print("Processing row %d" % ri)
    for ci in range(len(column_names)):
        data_dict[column_names[ci]].append(data_line[ci])
        
header_stud ="COD_FIS,GENERE,AA_IMM1,TIPO_TITOLO_SUP_DESC,PROVINCIA_SCUOLA_SIGLA,VOTO_MATURITA,VOTO_MIN,VOTO_MAX,STATO_STU_DESC,MOTIVO_STATO_STU_DESC,CDS_PROV,AA_ARRIVO_FISL,CDS_ARR,AA_PARTENZA_FISL,CDS_DESC_L,VOTO_FINALE,LODE_FLG,AA_ID"
colonne = header_stud.split(",")
studente={}
studente['MEDIA_VOTO']=[]
for ci in range(len(colonne)):
    studente[colonne[ci]]=[]
    
count=0
vuoti=0
#inizializza la sintesi dei dati imponendo che il numero della riga sia minore del numero totale di righe
while (count < 18936):
#Esiste un voto associato alla riga? Se sì: proseguo con l'analisi. Se no: passo alla riga successiva
    if(data_dict['VOTO'][count]):
#associo al valore 'temp' il rodotto del voto associato alla riga con i crediti associati alla riga
        crediti=int(data_dict['CREDITI'][count])
        temp=int(data_dict['VOTO'][count])*crediti
#inizializzo il contatore di ripetizioni per codice fiscale
        ro=1
#finchè il codice fiscale della riga è uguale a quello delle successive sommo i valori di 'temp' per ciascuna riga successiva ed i valori dei crediti totali
        while (data_dict["COD_FIS"][count]==data_dict["COD_FIS"][count+ro]):
#Esegue solo se esiste un valore di voto associato a ciascuna delle righe successive
            if(data_dict['VOTO'][count+ro]):
                temp=temp+(int(data_dict['VOTO'][count+ro])*int(data_dict['CREDITI'][count+ro]))
                crediti=crediti+int(data_dict['CREDITI'][count+ro])
            ro=ro+1
#Associo i valori del nuovo dizionario studente
        for ci in range(len(colonne)):
            studente[colonne[ci]].append(data_dict[colonne[ci]][count])
        studente['MEDIA_VOTO'].append(int(temp/crediti))
#stabilisco la nuova riga di partenza come quella per la quale non è più vero che si ripete lo stesso codice fiscale
        count=count+ro
    else:
#prima di passare alla riga successiva aggiungo una unità al numero di studenti che non hanno voti associati, se non, al massimo, una ideoneità linguistica
        if(count < 18935):
#considero i casi di idoneità linguistica ed evito che gli studenti vengano contati due volte
            if(data_dict["COD_FIS"][count]!=data_dict["COD_FIS"][count+1]): 
                vuoti=vuoti+1
                for ci in range(len(colonne)):
                    studente[colonne[ci]].append(data_dict[colonne[ci]][count])
                studente['MEDIA_VOTO'].append(int(temp/crediti))
        count=count+1

Processing row 18935


In [13]:
c=1
for ri in range(len(data_dict['COD_FIS'])-1):
    if data_dict['COD_FIS'][ri]!=data_dict['COD_FIS'][ri+1]: c=c+1
print(c)

1993


In [90]:
print(studente['GENERE'])

['F', 'M', 'M', 'M', 'M', 'M', 'M', 'F', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'F', 'M', 'M', 'M', 'F', 'F', 'M', 'M', 'M', 'F', 'F', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'F', 'F', 'M', 'M', 'M', 'M', 'F', 'F', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'F', 'F', 'F', 'M', 'M', 'M', 'M', 'M', 'M', 'F', 'F', 'F', 'M', 'M', 'F', 'F', 'M', 'M', 'F', 'M', 'M', 'M', 'M', 'F', 'M', 'F', 'F', 'M', 'M', 'M', 'F', 'M', 'M', 'M', 'F', 'M', 'M', 'F', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'F', 'M', 'F', 'F', 'M', 'F', 'M', 'M', 'M', 'F', 'M', 'F', 'M', 'F', 'M', 'M', 'M', 'M', 'M', 'F', 'M', 'M', 'M', 'M', 'M', 'F', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'F', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'F', 'F', 'M', 'M', 'M', 'M', 'M', 'F', 'M', 'M', 'F', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'F', 'F', 'M', 'M', 'F', 'M', 'F', 'M', 'M', 'M', 'F', 'M', 'M', 'F', 'F', 'M',